# Kernel time series prediction

This notebook generates reference results on all tree data sets using _Time Series Prediction for Graphs in Kernel and Dissimilarity Spaces_ ([Paaßen, Göpfert, and Hammer, 2018](https://arxiv.org/abs/1704.06498)). This method performs a prediction in kernel space as follows. Let $\{(x_i, y_i)\}_{i \in \{1, \ldots, m\}}$ be the training data where $y_i$ is the true successor of $x_i$. Then, the prediction $\phi(y)$ in kernel space for the input tree $x$ is given as

$$\phi(y) = \phi(x) + \sum_i \gamma_i \cdot [ \phi(y_i) - \phi(x_i) ]$$

where $\vec \gamma = \big( K + \sigma^2 \cdot I \big)^{-1} \cdot \vec k(x)$ are the coefficients computed by Gaussian process/kernel prediction, where $K$ is the kernel matrix between all training data and where $\vec k(x)$ is the vector of kernel values from $x$ to all training trees. The kernel we use here is $k(x, y) = \exp\big(-\frac{1}{2 \psi^2} \cdot d(x, y)^2 \big)$, where $d$ is the tree edit distance and $\psi$ is a hyper-parameter called bandwidth. This is the suggested kernel of ([Paaßen, Göpfert, and Hammer, 2018](https://arxiv.org/abs/1704.06498)).

To map back from kernel space to a tree we employ the scheme recommended by the authors in ([Paaßen et al., 2018](https://jedm.educationaldatamining.org/index.php/JEDM/article/view/158)). In particular, we greedily apply edits $\delta$ to $x$ that move towards training data trees $y_i$ with positive coefficients $\gamma_i$ or towards training data trees $x_i$ with negative coefficients $\gamma_i$ and which reduce the loss

$$\ell(\delta) = d(x, \delta(x))^2 + \sum_i \gamma_i [ d(\delta(x), y_i)^2 - d(\delta(x), x_i)^2) ]$$

until no edit $\delta$ is found anymore which reduces the loss.

For even more details, please refer to the source code at `kernel_time_series_prediction.py`.

## Boolean Formulae and Peano Addition

Refer to `boolean_formulae.py` and `peano_additon.py` for details on the datasets.

In [ ]:
# evaluate kernel tree time series prediction on both tasks in five repeats
import json
import os
import random
import time
import numpy as np
import edist.tree_utils as tu
import edist.tree_edits as te
import edist.ted as ted
import boolean_formulae
import peano_addition
from kernel_time_series_prediction import KernelTreePredictor

R = 1
# the number of time series we generate for training
M = 20
# the number of time series for testing
N_test = 10

# model hyper parameters
psi   = None
sigma = 1E-3

tasks = ['boolean', 'peano']

errors = np.zeros((len(tasks), R))
baseline_errors = np.zeros((len(tasks), R))
times  = np.zeros((len(tasks), R))
prediction_times  = np.zeros((len(tasks), R))

# iterate over all tasks
for task_idx in range(len(tasks)):
    task = tasks[task_idx]
    print('\n\n --- task %s --- \n' % task)
    if task == 'boolean':
        sampling_fun = boolean_formulae.generate_time_series
    else:
        sampling_fun = peano_addition.generate_time_series

    # do repeats
    for r in range(R):
        print('repeat %d of %d' % (r+1, R))
        # sample random training data
        training_data = []
        for i in range(M):
            print(f'sampling {i+1}/{M}')
            training_data.append(sampling_fun())
        # fit model
        start = time.time()
        model = KernelTreePredictor(psi = psi, sigma = sigma)
        print('fitting model.')
        model.fit(training_data)
        times[task_idx, r] = time.time() - start
        print('took %g seconds to train' % times[task_idx, r])
        # evaluate it on the test data
        rmse = 0.
        baseline_rmse = 0.
        m = 0
        for i in range(N_test):
            # sample test time series
            time_series = sampling_fun()
            for t in range(len(time_series)-1):
                nodes, adj = time_series[t][0], time_series[t][1]
                # predict next tree
                start = time.time()
                _, nodes_pred, adj_pred = model.predict(nodes, adj)
                prediction_times[task_idx, r] += time.time() - start
                # compare to actual next tree
                nodes_act, adj_act = time_series[t+1][0], time_series[t+1][1]
                d = ted.ted(nodes, adj, nodes_act, adj_act)
                baseline_rmse += d * d
                if nodes_pred != nodes_act or adj_pred != adj_act:
                    d = ted.ted(nodes_pred, adj_pred, nodes_act, adj_act)
                    rmse += d * d
            m += len(time_series)
        rmse = np.sqrt(rmse / m)
        baseline_rmse = np.sqrt(baseline_rmse / m)
        prediction_times[task_idx, r] /= m
        print('took %g seconds to predict' % prediction_times[task_idx, r])
        errors[task_idx, r] = rmse
        baseline_errors[task_idx, r] = baseline_rmse
        print('RMSE: %g versus baseline RMSE %g' % (rmse, baseline_rmse))

(['root', 'or', 'and', 'or', 'not_x', 'not_x', 'not_x', 'y'], [[1], [2, 7], [3, 6], [4, 5], [], [], [], []])


 --- task boolean --- 

repeat 1 of 1
sampling 1/20
sampling 2/20
sampling 3/20
sampling 4/20
sampling 5/20
sampling 6/20
sampling 7/20
sampling 8/20
sampling 9/20
sampling 10/20
sampling 11/20
sampling 12/20
sampling 13/20
sampling 14/20
sampling 15/20
sampling 16/20
sampling 17/20
sampling 18/20
sampling 19/20
sampling 20/20
fitting model.


100%|██████████| 20/20 [00:00<00:00, 20010.99it/s]

computing pairwis distances
